In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def init_two_layer_model(input_size, hidden_size, output_size):
    model = {}
    model['W1'] = 0.00001 * np.random.randn(input_size, hidden_size)
    model['b1'] = np.zeros(hidden_size)
    model['W2'] = 0.00001 * np.random.randn(hidden_size, output_size)
    model['b2'] = np.zeros(output_size)
    return model

In [3]:
def two_layer_net(X, model, y=None, reg=0.0, verbose=False):
    # unpack variables from the model dictionary
    W1,b1,W2,b2 = model['W1'], model['b1'], model['W2'], model['b2']
    N, D = X.shape

    # compute the forward pass
    scores = None
    # W1:4*10 b1:10
    print("x.shape:", X.shape)
    print('w1.shape:', W1.shape)
    i1 = X.dot(W1) + b1
    # i1 5*10
    o1 = np.where(i1> 0, i1, 0) # Otherwise causing problems with the shape
    # 5*10 dot 10*3
    scores = o1.dot(W2) + b2
    print('scores.shape:', scores.shape)
    # If the targets are not given then jump out, we're done
    if y is None:
        print('y is None, scores:')
        return scores

    # compute the loss
    loss = None
    f = scores.T - np.max(scores, axis =1) # Regularizing
    f = np.exp(f)
    p = f / np.sum(f, axis=0)

    # loss function
    distr = np.arange(N) # random distribution
    loss = np.mean(-np.log(p[y, distr]))
    loss += (0.5*reg) * np.sum(W1 * W1)
    loss += (0.5*reg) * np.sum(W2 * W2)

    # compute the gradients
    grads = {}

    df = p  # (C, N)
    df[y, distr] -= 1
    # (H, C) = ((C, N) x (N, H)).T
    dW2 = df.dot(o1).T / N  # (H, C)
    dW2 += reg * W2
    grads['W2'] = dW2

    # C = (C, N)
    db2 = np.mean(df, axis=1)  # C
    grads['b2'] = db2

    # (N, H) =  (H, C)
    dresp1 = W2.dot(df).T / N
    ds1 = np.where(i1 > 0, dresp1, 0)  # (N, H)
    dW1 = X.T.dot(ds1)  # (D, H)
    dW1 += reg * W1
    grads['W1'] = dW1

    db1 = np.sum(ds1, axis=0)  # H
    grads['b1'] = db1
    return loss, grads

In [4]:
input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5

In [5]:
#  build the initial model
def init_toy_model():
    model = {}
    model['W1'] = np.linspace(-0.2, 0.6, num=input_size * hidden_size).reshape(input_size, hidden_size)
    model['b1'] = np.linspace(-0.3, 0.7, num=hidden_size)
    model['W2'] = np.linspace(-0.4, 0.1, num=hidden_size * num_classes).reshape(hidden_size, num_classes)
    model['b2'] = np.linspace(-0.5, 0.9, num=num_classes)
    return model

In [6]:
#  contruct the used input data
# 5 * 4
def init_toy_data():
    x = np.linspace(-0.2, 0.5, num=num_inputs*input_size).reshape(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    print('input x:',x)
    return x, y

In [7]:
# 4 * 10 * 3
model = init_toy_model()
x, y = init_toy_data()

input x: [[-0.2        -0.16315789 -0.12631579 -0.08947368]
 [-0.05263158 -0.01578947  0.02105263  0.05789474]
 [ 0.09473684  0.13157895  0.16842105  0.20526316]
 [ 0.24210526  0.27894737  0.31578947  0.35263158]
 [ 0.38947368  0.42631579  0.46315789  0.5       ]]


In [8]:
y2 = np.array([1, 2, 3])
scores = two_layer_net(x, model, y2,verbose=True)
print(scores)

x.shape: (5, 4)
w1.shape: (4, 10)
scores.shape: (5, 3)


IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (3,) (5,) 